## IMPORT LIBRARIES

In [69]:
import pandas as pd
import json
import re
from unidecode import unidecode
import nltk

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [137]:
from nltk.corpus import stopwords
from wordcloud import WordCloud


## LOAD BASES

In [73]:
path = "../data/data_music_clean.json"
f = open(path)
data_json = json.load(f)
data_json.keys()

dict_keys(['17 Léguas e Meia', '2001', 'Abri a Porta', 'A Bruxa de Mentira', 'Acertei No Milhar', 'A Ciência Em Si', 'A Coisa Mais Linda Que Existe', 'A Debandada', 'Afoxé É', 'A Gaivota', 'Água Benta', 'Água de Meninos', 'A Linha e o Linho', 'A Luta Contra a Lata ou A Falência do Café', 'A Mão da Limpeza', 'Amarra o Teu Arado a Uma Estrela', 'Amor até o fim', 'A Moreninha', 'Andar Com Fé', 'A Novidade', 'Aos pes da cruz', 'A Paz', 'Aquele Abraço', 'Aquele Abraço ( Acoustic)', 'Aqui e Agora', 'A Raça Humana', 'A Rua', 'Asa Branca', 'A Situação Do Escurinho', 'A Sociedade Afluente', 'Assum Preto', 'Átimo de Pó', 'A Última Coisa Bonita', 'A Volta da Asa Branca', 'A Voz do Vivo', 'Axé, Babá', 'Babá Alapalá', 'Baby Hippie', 'Back in Bahia', 'Back in Bahia (Amor e Sorte)', 'Back In Bahia / Atrás do Trio Elétrico', 'Baião atemporal', 'Baião da Penha', 'Balafon', 'Banda Um', 'Barca Grande', 'Beira-Mar', 'Beira-Mar (Acoustic)', 'Buda Nagô', 'Caçada das Leoas', 'Cada Macaco no Seu Galho', 'Cada

In [129]:
base_pos = pd.DataFrame.from_dict(data_json, orient='index')
base_pos.reset_index(inplace=True)
base_pos.columns = ["Musica", "Letra"]
print(base_pos.shape)
base_pos.head()

(323, 2)


,Musica,Letra
0,17 Léguas e Meia,eu já andei sem parar dezessete légua e meia p...
1,2001,astronauta libertado minha vida me ultrapassa ...
2,Abri a Porta,abri a porta apareci a mais bonita sorriu pra ...
3,A Bruxa de Mentira,a bruxa de mentira bombom de rapadura saborosa...
4,Acertei No Milhar,"etelvina, acertei no milhar ganhei quinhentos ..."


In [130]:
base_pos.isnull().sum()

Musica    0
Letra     0
dtype: int64

In [131]:
## Check if we have duplicated lyrics

filtro_duplicated = base_pos.duplicated(subset=["Letra"], keep="first")
base_pos[filtro_duplicated]["Letra"].values

array(['essa ladeira que ladeira é essa? essa é a ladeira da preguiça preguiça que eu tive sempre de escrever para a família e de mandar contar pra casa que esse mundo é uma maravilha e pra saber se a menina já conta as estrelas e sabe a segunda cartilha e pra saber se o menino já canta cantigas e já não bota mais a mão na barguilha e pra falar do mundo, falar uma besteira formenteira é uma ilha onde se chega de barco, mãe que nem lá na ilha do medo que nem lá na ilha do frade que nem lá na ilha de maré que nem lá salina das margaridas essa ladeira que ladeira é essa? essa é a ladeira da preguiça ela não é de hoje ela é desde quando se amarrava cachorro com linguiça'],
      dtype=object)

In [132]:
## Lets exclude the duplicated rows 

base_pos = base_pos[~filtro_duplicated]
print(base_pos.shape)
base_pos.head(10)

(322, 2)


,Musica,Letra
0,17 Léguas e Meia,eu já andei sem parar dezessete légua e meia p...
1,2001,astronauta libertado minha vida me ultrapassa ...
2,Abri a Porta,abri a porta apareci a mais bonita sorriu pra ...
3,A Bruxa de Mentira,a bruxa de mentira bombom de rapadura saborosa...
4,Acertei No Milhar,"etelvina, acertei no milhar ganhei quinhentos ..."
5,A Ciência Em Si,se toda coincidência tende a que se entenda e ...
6,A Coisa Mais Linda Que Existe,coisa mais linda nesse mundo é sair por um seg...
7,A Debandada,aah! yona yona yona yona yona yona yona yona y...
8,Afoxé É,"êô, êô êô, êô é bom pa ioiô (é bom pa ioiô) é ..."
9,A Gaivota,gaivota menina de asas paradas voando no sonho...


In [133]:
stopwords_list = stopwords.words('portuguese')

In [134]:
def stop_words(no_punctuation):
    text_list = no_punctuation.split(' ')
    list_text_nostopword = []
    for word in text_list:
        if word not in stopwords_list:
            list_text_nostopword.append(word)

    delimiter = " "  # Delimiter to use between elements

    result_string = delimiter.join(list_text_nostopword)
    return result_string

# stop_words(no_punctuation)

In [135]:
# list_text_nostopword = ""
# for word in text_list:
#     if word not in stopwords_list:
#         list_text_nostopword += word + " "

# list_text_nostopword.strip()

In [136]:
def normalize_texts(text):
    lower = text.lower()
    NON_ALPHANUM = re.compile(r'[\W]')
    no_punctuation = NON_ALPHANUM.sub(r' ', lower)
    text_nostopword = stop_words(no_punctuation)
    normalized_text = unidecode(text_nostopword)
    return normalized_text

# text0 = base_pos["Letra"].values[240]
# normalize_texts(text0)

base_pos["Letra_Normalized"] = base_pos["Letra"].apply(lambda x: normalize_texts(x))
print(base_pos.shape)
base_pos.head(10)

(322, 3)


,Musica,Letra,Letra_Normalized
0,17 Léguas e Meia,eu já andei sem parar dezessete légua e meia p...,andei parar dezessete legua meia pra ir forro ...
1,2001,astronauta libertado minha vida me ultrapassa ...,astronauta libertado vida ultrapassa qualquer ...
2,Abri a Porta,abri a porta apareci a mais bonita sorriu pra ...,abri porta apareci bonita sorriu pra mim naque...
3,A Bruxa de Mentira,a bruxa de mentira bombom de rapadura saborosa...,bruxa mentira bombom rapadura saborosa figura ...
4,Acertei No Milhar,"etelvina, acertei no milhar ganhei quinhentos ...",etelvina acertei milhar ganhei quinhentos con...
5,A Ciência Em Si,se toda coincidência tende a que se entenda e ...,toda coincidencia tende entenda toda lenda que...
6,A Coisa Mais Linda Que Existe,coisa mais linda nesse mundo é sair por um seg...,coisa linda nesse mundo sair segundo encontrar...
7,A Debandada,aah! yona yona yona yona yona yona yona yona y...,aah yona yona yona yona yona yona yona yona y...
8,Afoxé É,"êô, êô êô, êô é bom pa ioiô (é bom pa ioiô) é ...",eo eo eo eo bom pa ioio bom pa ioio bom pa...
9,A Gaivota,gaivota menina de asas paradas voando no sonho...,gaivota menina asas paradas voando sonho d agu...


In [140]:
all_words = [text for text in base_pos["Letra_Normalized"]]

delimiter = " "  # Delimiter to use between elements

all_words_string = delimiter.join(all_words)
all_words_string

'andei parar dezessete legua meia pra ir forro dancar ai  ai  ai  ai  ai  ai forro dancar valeu pena andar dezessete legua meia pois rosinha tava la ai  ai  ai  ai  ai  ai rosinha tava la cheguei forro moido  carne viva lapada logo socorrido tomando tres talagada ze sanfoneiro gemeu fole baiao rodei rosa terreiro  rocando coracao rodei rosa terreiro  rocando coracao ai  ai  ai tentacao astronauta libertado vida ultrapassa qualquer rota faca dei grito escuro parceiro futuro reluzente galaxia quase posso palpar vida grita emprenha reproduz velocidade luz cor ceu compoe mar azul dissolve equacao propoe computador resolve  amei velocidade casei sete planetas filho  cor espaco faco rota anoluz calculo dentro passo dor cicatriz morte quis  bracos dois mil anos nasci ter idade casado solteiro baiano estrangeiro sangue gasolina correndo magoa peito sal fruta fervendo copo d agua abri porta apareci bonita sorriu pra mim naquele instante convenci bom vida vai prosseguir vai prosseguir vai da pra

In [142]:
word_cloud = WordCloud(width= 800, height= 500,
                        max_font_size = 110,
                        collocations = False).generate(all_words_string)
# Plotando nuvem de palavras
plt.figure(figsize=(24,12))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

ValueError: Only supported for TrueType fonts

In [2]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Sample text data
text_data = "This is a sample text data for generating a word cloud. Word clouds are fun and informative visualizations."

# Create a WordCloud object
wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(text_data)

# Display the word cloud using matplotlib
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

ValueError: Only supported for TrueType fonts

### TAREFAS:
    - Criar um codigo que exclui as cifras
    - Tirar possiveis espacos no comeco e no fim das letras. Usar strip()
    - Tirar pontuacoes e converter tudo para minusculo
    - Checar valores duplicados
    - Criar funcao para exlcuir letras em ingles

In [10]:
## remove symbols

## Regra 1: verifica por um numero em posição j e verifica por espaço vazio ou \n em posicao i, remove caracteres de i a j
## Regra 1 falhou pois nem todas as cifras começam por numero

## Regra 2: verifica por linha com maior numero de caracteres numericos, exclui linha

def remover_cifras(letra_com_cifras):
    pass


remover_cifras(letra_com_cifras="A12 \n1seefasar 8r238r 23bq")

In [93]:
## Regra 2: verifica linha a linha e verifica se há algum numero
## Regra 2 falhou pois nem todos os acordes tem numeros, mas consigo separar linha a linha

letra_com_cifras="A12 \n1seefasar\n 8r238r 23bq"

## Identificar todas as localizações de \n no texto

def index_novalinha(texto):
    quebras_de_linha = [index for index, caractere in enumerate(texto) if caractere == '\n']
    return quebras_de_linha

## verifica entre as linhas se há algum numero e retorna

def entre_linhas(texto, quebras_de_linha):
    quebras_de_linha.append(len(texto))
    # exclui a sequencia que contem numeral
    start = 0
    for pos in quebras_de_linha:
        texto_novo = texto[start:pos]
        start = pos + 1
        print(texto_novo)
        

texto = "13jk1\n123121j\nhn23ujnb1u\n1231\nnj"
quebras = index_novalinha(texto)

print(entre_linhas(texto, quebras))

13jk1
123121j
hn23ujnb1u
1231
nj
None


In [22]:
letra_exemplo = base_pos.iloc[19,1]
letra_exemplo = remove_expr(letra_exemplo)
letra_exemplo.split('\n')

['E6',
 'Andar com fé eu vou',
 'B7 b7/4 b7',
 'Que a fé não \'causetuma "faiá"',
 'E6',
 'Andar com fé eu vou',
 'B7 b7/4 b7',
 'Que a fé não \'causetuma "faiá"',
 'E6',
 'Andar com fé eu vou',
 'B7 b7/4 b7',
 'Que a fé não \'causetuma "faiá"',
 'E6',
 'Que a fé tá na mulher',
 'B7',
 'A fé tá na cobra coral',
 'A',
 'Ôô',
 'G#m7 F#m7 B7',
 'Num pedaço de pão',
 'E6',
 'A fé tá na maré',
 'B7',
 'Na lâmina de um punhal',
 'A',
 'Ôô',
 'G#m7 F#m7 B7/4 B7',
 'Na luz, na escuridão',
 'E6',
 'Andá com fé eu vou',
 'B7 b7/4 b7',
 'Que a fé não \'causetuma "faiá"',
 'E6',
 'Andá com fé eu vou',
 'B7 b7/4 b7',
 'Que a fé não \'causetuma "faiá"',
 'E6',
 'A fé tá na manhã',
 'B7',
 'A fé tá no anoitecer',
 'A',
 'Ôô',
 'G#m7 F#m7 B7/4 B7',
 'No calor do verão',
 'E6',
 'A fé tá viva e sã',
 'B7',
 'A fé também tá pra morrer',
 'E6',
 'Ôô',
 'G#m7 F#m7 B7/4 B7',
 'Triste na solidão',
 'E6',
 'Andá com fé eu vou',
 'B7 b7/4 b7',
 'Que a fé não \'causetuma "faiá"',
 'E6',
 'Andá com fé eu vou',


In [24]:
## Regra 3: listar alguns acordes, verificar cada linha e exclui linhas com maior numero de acordes

## obtem exemplo especifico e remove expressões

letra_exemplo = base_pos.iloc[19,1]
letra_exemplo = remove_expr(letra_exemplo)


##------------------------------------------------------
## definir função

def remover_cifras(letra):
    ## Lista de acordes
    basic_chords = [
        #'C', 'D', 'E', 'F', 'G', 'A', 'B',
        'Cm', 'Dm', 'Fm', 'Gm', 'Bm',
        'C7', 'D7', 'E7', 'F7', 'G7', 'A7', 'B7',
        'Cm7', 'Dm7', 'Em7', 'Fm7', 'Gm7', 'Am7', 'Bm7',
        'C6', 'D6', 'E6', 'F6', 'G6', 'A6', 'B6',
        'Cm6', 'Dm6', 'Em6', 'Fm6', 'Gm6', 'Am6', 'Bm6',
        'C6/4', 'D6/4', 'E6/4', 'F6/4', 'G6/4', 'A6/4', 'B6/4',
        'Cm6/4', 'Dm6/4', 'Em6/4', 'Fm6/4', 'Gm6/4', 'Am6/4', 'Bm6/4',
        'C#m', 'D#m', 'F#m', 'G#m', 'A#m',
        'C#m7', 'D#m7', 'F#m7', 'G#m7', 'A#m7',
        'C#m6', 'D#m6', 'F#m6', 'G#m6', 'A#m6',
        'C#m6/4', 'D#m6/4', 'F#m6/4', 'G#m6/4', 'A#m6/4',
        'C7/4', 'D7/4', 'E7/4', 'F7/4', 'G7/4', 'A7/4', 'B7/4',
        'Cm7/4', 'Dm7/4', 'Em7/4', 'Fm7/4', 'Gm7/4', 'Am7/4', 'Bm7/4',
    ]
    ## separa as linhas em elementos de uma lista
    letra_exemplo_separada = letra.split('\n')

    ## for loop aninhado para verificar cada item da lista de acordes 
    for chord in basic_chords:
        for index,linha in enumerate(letra_exemplo_separada):
            if (chord in linha) or (len(linha)==1):
                #letra_exemplo_separada[index] += '<<>>' # adiciona um sinal de identificação
                del letra_exemplo_separada[index]

    return letra_exemplo_separada

letra_exemplo_separada = remover_cifras(letra_exemplo)
letra_exemplo_separada

['Andar com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'Andar com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'Andar com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'Que a fé tá na mulher',
 'A fé tá na cobra coral',
 'Ôô',
 'Num pedaço de pão',
 'A fé tá na maré',
 'Na lâmina de um punhal',
 'Ôô',
 'Na luz, na escuridão',
 'Andá com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'Andá com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'A fé tá na manhã',
 'A fé tá no anoitecer',
 'Ôô',
 'No calor do verão',
 'A fé tá viva e sã',
 'A fé também tá pra morrer',
 'Ôô',
 'Triste na solidão',
 'Andá com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'Andá com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'Andá com fé eu vou',
 'Que a fé não \'causetuma "faiá"',
 'Certo ou errado até',
 'A fé vai onde quer que eu vá',
 'Ôô',
 'A pé ou de avião',
 'Mesmo a quem não tem fé',
 'A fé costuma acompanhar',
 'Ôô']

In [28]:
## une todas as letras em uma string
def string_chain(text):    
    lyrics = ''
    for element in text:
        lyrics += element + " "
        
    ## lower case
    lyrics = lyrics.lower()

    ## strip whitespaces
    lyrics = lyrics.strip()

    return lyrics

lyrics = string_chain(letra_exemplo_separada)
lyrics

'andar com fé eu vou que a fé não \'causetuma "faiá" andar com fé eu vou que a fé não \'causetuma "faiá" andar com fé eu vou que a fé não \'causetuma "faiá" que a fé tá na mulher a fé tá na cobra coral ôô num pedaço de pão a fé tá na maré na lâmina de um punhal ôô na luz, na escuridão andá com fé eu vou que a fé não \'causetuma "faiá" andá com fé eu vou que a fé não \'causetuma "faiá" a fé tá na manhã a fé tá no anoitecer ôô no calor do verão a fé tá viva e sã a fé também tá pra morrer ôô triste na solidão andá com fé eu vou que a fé não \'causetuma "faiá" andá com fé eu vou que a fé não \'causetuma "faiá" andá com fé eu vou que a fé não \'causetuma "faiá" certo ou errado até a fé vai onde quer que eu vá ôô a pé ou de avião mesmo a quem não tem fé a fé costuma acompanhar ôô'

In [29]:
## remove english lyrics

## load word list (370k)

path_dict = "../data/words_dictionary.json"
f_dict = open(path_dict)
data_dict = json.load(f_dict)

dict_en = data_dict.keys()

dict_en = list(dict_en)
# dict_en

In [30]:
## Regra: retirar letras com 25% das palavras em ingles
def split_lyrics(lyrics):
    lyrics_words = lyrics.split(' ')
    return lyrics_words

In [31]:
## funcao que retorna int para num itens iguais

def contar_itens_iguais(lista1, lista2):
    contagem = {}
    
    for item in lista1:
        if item not in contagem:
            contagem[item] = 0
        contagem[item] += 1
    
    itens_iguais = 0
    for item in lista2:
        if item in contagem and contagem[item] > 0:
            itens_iguais += 1
            contagem[item] -= 1
            
    return itens_iguais

def count_words(lyrics):    
    WORDS_EN_PERCENTUAL = 0.30

    words_lyrics = split_lyrics(lyrics)
    num_eq_words = contar_itens_iguais(words_lyrics, dict_en)

    return num_eq_words > int(len(words_lyrics)*WORDS_EN_PERCENTUAL)

In [32]:
## testar o preprocessamento para todas as letras

master_dict = {}
count = 0
for index in range(len(base_pos)):
    letra_exemplo = base_pos.iloc[index,1]
    letra_exemplo = remove_expr(letra_exemplo)
    letra_exemplo = remover_cifras(letra_exemplo)
    lyrics = string_chain(letra_exemplo)
    #if count_words(lyrics) == False:
    #   count += 1
    if count_words(lyrics) == False:
        master_dict[base_pos.iloc[index,0]] = lyrics

# master_dict


{'17 Léguas e Meia': 'eu já andei sem parar dezessete légua e meia pra ir no forró dançar ai, ai, ai, ai, ai, ai eu fui no forró dançar valeu a pena eu andar dezessete légua e meia pois rosinha tava lá ai, ai, ai, ai, ai, ai a rosinha tava lá cheguei no forró moído, em carne viva lapada mas logo fui socorrido tomando três talagada e quando zé sanfoneiro gemeu no fole o baião rodei por rosa o terreiro, roçando em meu coração rodei por rosa o terreiro, roçando em meu coração ai, ai, ai que tentação',
 '2001': "astronauta libertado minha vida me ultrapassa em qualquer rota que eu faça dei um grito no escuro sou parceiro do futuro na reluzente galáxia eu quase posso palpar a minha vida que grita emprenha e se reproduz na velocidade da luz a cor do céu me compõe mar azul me dissolve a equação me propõe computador me resolve  amei a velocidade casei com sete planetas por filho, cor e espaço não me tenho nem me faço a rota do anoluz calculo dentro do passo minha dor é cicatriz minha morte não

In [122]:
# Save the dictionary as a JSON 

with open('../data/data_music_clean.json', "w") as json_file:
    json.dump(master_dict, json_file, indent=4)